# Everything starts here

In [1]:
import numpy as np
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
import utils
from model import MobileNetFC as HerniaModel

## 1. Setting

### 1.1 Model parameters

In [3]:
# rebuild model
model_parameters = ('mobilenetfc', utils.num_classes, True, False)

### 1.2 Data augmentation

In [ ]:
# data augmentation and normalization for training
# just normalization for validation
data_transforms = {
    'training': transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'validation': transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

## 2. CNN (backbone) finetuning

### 2.0 Reload model

In [4]:
model = HerniaModel(*model_parameters).to(utils.device)

In [5]:
# skip lstm
model.skip_temp_fc = True

### 2.1 Set training hyperparameters

In [7]:
# training parameters
LEARNING_RATE = 0.004
EPOCHS = 4
BATCH_SIZE = 128
MOMENTUM = 0.9
GAMMA = 0.5
STEP_SIZE = 1

In [8]:
# criterion is cross entropy loss
criterion = nn.CrossEntropyLoss()

# observe that all parameters are being optimized
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)

# decay LR by a factor GAMMA every STEP_SIZE epochs
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

In [9]:
# create pytorch datasets
datasets = {x: utils.HernitiaDataset(utils.dfs_path + '/' + x + '_no_temp_os.pkl', data_transforms[x])  for x in ['training', 'validation']}

In [10]:
# instantiate data loaders
dataloaders = {x: utils.DataLoader(dataset=datasets[x], batch_size=BATCH_SIZE, shuffle=True) for x in ['training', 'validation']}

### 2.2 Train

In [11]:
utils.train_model(model = model, 
                    model_name = model.model_name,  #  name of the model which will be the name of the saved weights file within /weights
                    dataloaders = dataloaders, 
                    criterion = criterion, 
                    optimizer = optimizer, 
                    scheduler = exp_lr_scheduler, 
                    num_epochs=EPOCHS)

Epoch 1/4
----------
610596/610596: [===============================>] - ETA 3.1sssss
training Loss: 0.1746 Acc: 0.9407
50216/50216: [===============================>] - ETA 2.3sss
validation Loss: 0.7831 Acc: 0.7957
Epoch 2/4
----------
610596/610596: [===============================>] - ETA 0.8ssss
training Loss: 0.0410 Acc: 0.9870
50216/50216: [===============================>] - ETA 0.4sss
validation Loss: 0.9177 Acc: 0.8001
Epoch 3/4
----------
 20352/610596: [=>..............................] - ETA 3610.0s

KeyboardInterrupt: 

## 3. Temp FC training

### 3.0 Reload model

In [42]:
# rebuild model
model = HerniaModel(*model_parameters).to(utils.device)

In [43]:
# skip lstm
model.skip_temp_fc = True

In [44]:
# reload weights from finetuning
model.load_state_dict(torch.load(utils.weights_path + '/' + model.model_name + '.pkl'))

<All keys matched successfully>

### 3.1 Set training hyperparameters

In [47]:
# training parameters
LEARNING_RATE = 0.001
EPOCHS = 20
MOMENTUM = 0.9
GAMMA = 0.75
STEP_SIZE = 2

In [48]:
# criterion is cross entropy loss
criterion = nn.CrossEntropyLoss()

# observe that all parameters are being optimized
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)

# decay LR by a factor GAMMA every STEP_SIZE epochs
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

In [49]:
# create pytorch datasets
datasets = {x: utils.HernitiaDataset(utils.dfs_path + '/' + x + '_temp.pkl', data_transforms[x])  for x in ['training', 'validation']}

In [50]:
# instantiate data loaders
dataloaders = {x: utils.DataLoader(dataset=datasets[x], batch_size=BATCH_SIZE, shuffle=False) for x in ['training', 'validation']}

### 4.3 Train

In [51]:
utils.evaluate_model(model, dataloaders['validation'], criterion)

50464/50464: [===============================>] - ETA 0.1sss
Loss: 0.6794 Acc: 0.8016


In [52]:
utils.train_model(model = model, 
                    model_name = model.model_name,  #  name of the model which will be the name of the saved weights file within /weights
                    dataloaders = dataloaders, 
                    criterion = criterion, 
                    optimizer = optimizer, 
                    scheduler = exp_lr_scheduler, 
                    num_epochs=EPOCHS)

Epoch 1/20
----------
165728/165728: [===============================>] - ETA 0.2ssss
training Loss: 1.2784 Acc: 0.5693
50464/50464: [===============================>] - ETA 0.1sss
validation Loss: 2.2669 Acc: 0.2251
Epoch 2/20
----------
165728/165728: [===============================>] - ETA 0.2sss
training Loss: 0.7686 Acc: 0.7390
50464/50464: [===============================>] - ETA 0.1sss
validation Loss: 2.5115 Acc: 0.1373
Epoch 3/20
----------
165728/165728: [===============================>] - ETA 0.2sss
training Loss: 0.5826 Acc: 0.8057
50464/50464: [===============================>] - ETA 0.1sss
validation Loss: 2.8388 Acc: 0.1324
Epoch 4/20
----------
165728/165728: [===============================>] - ETA 0.2sss
training Loss: 0.4888 Acc: 0.8369
50464/50464: [===============================>] - ETA 0.1sss
validation Loss: 3.1000 Acc: 0.1226
Epoch 5/20
----------
165728/165728: [===============================>] - ETA 0.2ssss
training Loss: 0.3884 Acc: 0.8743
50464/50464: [=

## 5. Whole network finetuning

### 5.0 Reload model

In [39]:
# rebuild model
model = HerniaModel(*model_parameters).to(utils.device)

In [40]:
# reload weights from finetuning
model.load_state_dict(torch.load(utils.weights_path + '/' + model.model_name + '.pkl'))

<All keys matched successfully>

### 5.1 Set training hyperparameters

In [42]:
# training parameters
LEARNING_RATE = 0.0008
EPOCHS = 3
BATCH_SIZE = 32
MOMENTUM = 0.9
GAMMA = 0.3
STEP_SIZE = 1

In [43]:
# criterion is cross entropy loss
criterion = nn.CrossEntropyLoss()

# observe that all parameters are being optimized
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)

# decay LR by a factor GAMMA every STEP_SIZE epochs
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

In [44]:
# create pytorch datasets
datasets = {x: utils.HernitiaDataset(utils.dfs_path + '/' + x + '_temp.pkl', data_transforms[x])  for x in ['training', 'validation']}

In [45]:
# instantiate data loaders
dataloaders = {x: utils.DataLoader(dataset=datasets[x], batch_size=BATCH_SIZE, shuffle=False) for x in ['training', 'validation']}

### 5.2 Train

In [46]:
utils.train_model(model = model, 
                    model_name = model.model_name,  #  name of the model which will be the name of the saved weights file within /weights
                    dataloaders = dataloaders, 
                    criterion = criterion, 
                    optimizer = optimizer, 
                    scheduler = exp_lr_scheduler, 
                    num_epochs=EPOCHS)

Epoch 1/3
----------
165728/165728: [===============================>] - ETA 0.2ssss
training Loss: 1.8947 Acc: 0.3164
50464/50464: [===============================>] - ETA 0.1sss
validation Loss: 2.0512 Acc: 0.2456
Epoch 2/3
----------
165728/165728: [===============================>] - ETA 0.2sss
training Loss: 1.4319 Acc: 0.5317
50464/50464: [===============================>] - ETA 0.1sss
validation Loss: 1.8941 Acc: 0.2812
Epoch 3/3
----------
  3744/165728: [>...............................] - ETA 947.6s

KeyboardInterrupt: 

## 6. Make Kaggle prediction

In [ ]:
# rebuild model
model = HerniaModel(model_name = 'mobilenet_v2_lstm', 
                    num_classes = utils.num_classes, 
                    pretrained = True, 
                    num_layers_lstm = 2,
                    bidirectional = False,
                    hidden_size_lstm = 32,
                    skip_lstm = False).to(utils.device)

In [ ]:
# reload weights from finetuning
model.load_state_dict(torch.load(utils.weights_path + '/' + model.model_name + '.pkl'))

In [ ]:
# data augmentation and normalization for training
testing_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [ ]:
utils.predict_kaggle(model = model, 
                    model_name = model.model_name, # name of the model from which to load the weights within weights/
                    transform = testing_transforms, 
                    predictions_name = model.model_name,
                    batch_size = BATCH_SIZE) # name of the csv file to which the predictions are saved within predictions/

## Bonus. Evaluate model

In [20]:
model = HerniaModel(*model_parameters).to(utils.device)

In [29]:
# skip lstm
model.skip_temp_fc = True

In [30]:
# reload weights from finetuning
model.load_state_dict(torch.load(utils.weights_path + '/' + model.model_name + '.pkl'))

<All keys matched successfully>

In [36]:
BATCH_SIZE = 32

# criterion is cross entropy loss
criterion = nn.CrossEntropyLoss()

In [37]:
# just normalization for validation
data_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [38]:
# create pytorch datasets
dataset = utils.HernitiaDataset(utils.dfs_path + '/' + 'validation' + '_lstm.pkl', data_transform)

In [39]:
# instantiate data loaders
dataloader = utils.DataLoader(dataset=dataset, batch_size=BATCH_SIZE, shuffle=False)

In [40]:
utils.evaluate_model(model, dataloader, criterion)

50464/50464: [===============================>] - ETA 0.1sss
Loss: 0.6794 Acc: 0.8016


In [93]:
input, labels = next(iter(dataloader))

In [94]:
input = input.to(utils.device)

In [95]:
outputs = model(input)

RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 14.76 GiB total capacity; 13.45 GiB already allocated; 15.75 MiB free; 13.56 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [89]:
torch.max(outputs, 1)

NameError: name 'outputs' is not defined